In [390]:
import pandas as pd
from icecream import ic
import numpy as np
import random
from itertools import combinations
from geopy.distance import geodesic
from dataclasses import dataclass
#from itertools import iterrows
from matplotlib import pyplot as plt
from itertools import accumulate
from tqdm.auto import tqdm

cities= pd.read_csv('cities/vanuatu.csv', header=None, names=['name', 'lat', 'lon'])
cities


,name,lat,lon
0,Isangel,-19.53,169.28
1,Lakatoro,-16.09,167.40
2,Longana,-15.30,168.00
3,Luganville,-15.51,167.15
4,Norsup,-16.07,167.39
5,Port Olry,-15.05,167.05
6,Sola,-13.87,167.55
7,Vila,-17.74,168.31


In [378]:

DIST_MATRIX= np.zeros((len(cities), len(cities)))
for c1, c2 in combinations(cities.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km


for c1 in range(len(cities)):
    DIST_MATRIX[c1,c1]=np.inf



masked_dist_matrix = np.ma.masked_equal(DIST_MATRIX, np.inf)
min_index = np.ma.argmin(masked_dist_matrix[0])



#np.argmin(DIST_MATRIX[0])

cities.head()


,name,lat,lon
0,Isangel,-19.53,169.28
1,Lakatoro,-16.09,167.40
2,Longana,-15.30,168.00
3,Luganville,-15.51,167.15
4,Norsup,-16.07,167.39


In [379]:
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(cities)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

In [380]:
def startgreedy(start):   #greedy approach that takes always the nearest city
   giro=[]
   costi=[]
   startpoint=globalstart=posto= start #np.int64(np.random.randint(len(cities)))
   giro.append(startpoint)
   copydist=DIST_MATRIX.copy()
   distances = DIST_MATRIX.copy()
   distances[:, posto] = np.inf
      
   for i in range(len(cities)):
      if(i==(len(cities))-1):
         posto=globalstart
      else:  
         posto=np.argmin(distances[startpoint])

      costo=copydist[posto][startpoint]
      costi.append(costo)

      giro.append(posto)
      distances[:, posto] = np.inf
      startpoint=posto

      

   return(giro)






In [381]:





def startexp3(startpoint,distances,ended,near,giro):   #trying to start in 2 different point so that alghorithm will up in the middle 
    nearest_index=0
    if(ended==True):
        nearest_index=2
    
    second_near_index=nearest_index+1   
    near[nearest_index]=np.ma.argmin(distances[startpoint]) #nearest 
    distances[startpoint,near[nearest_index]] = np.ma.masked  #nearest to origin
    distances[near[nearest_index],startpoint] = np.ma.masked
    near[second_near_index]=np.ma.argmin(distances[:,startpoint])

    

    distances[startpoint] = np.ma.masked
    distances[:, startpoint] = np.ma.masked
    distances[startpoint, near[second_near_index]] = np.ma.masked
    distances[near[second_near_index], startpoint] = np.ma.masked
    giro[near[second_near_index]] = startpoint
    giro[startpoint] = near[nearest_index]
    distances[near[second_near_index]]=np.ma.masked  # none can start from the second near 
    distances[:,near[nearest_index]]=np.ma.masked  # none can get to the  nearest 



In [382]:



def findNearToNear(near,distances,giro):   #each open end of the path looks for the nearest element 
   togo=0
   count =len(giro)
   for e in giro:
      if e ==-1:
         togo+=1
   blocked=[]
   startvalue=-1-len(giro)

   neartonear=[startvalue,startvalue,startvalue,startvalue]
   n=0

   while n<3:
      onlybranch=0
      count=0
      if(n==2):
         count = np.count_nonzero(giro != -1)
      if(near[0+n]>=0 and near[1+n] >=0 ):

         distances[near[0+n],near[1+n]] = distances[near[1+n],near[0+n]]= np.ma.masked
 
      if near[0+n] >=0 and (not distances[near[0+n]].mask.all()):
         
         neartonear[0+n]=np.ma.argmin(distances[near[0+n]])


      else:
         onlybranch+=1
 

      if   near[1+n] >=0 and  not distances[:,near[1+n]].mask.all():
         neartonear[1+n]=np.ma.argmin(distances[:,near[1+n]])
        
      else:
         onlybranch+=2

      val=0
      if(onlybranch==0 and neartonear[1+n]==neartonear[n]):
         dist2=distances[near[1+n],neartonear[1+n]]
         dist1=distances[near[n],neartonear[n]]
         if(dist2>dist1):
            val=0
         else:
            val=1
      if(onlybranch!= 3 and onlybranch!=1):
         if (distances.mask[neartonear[n+val],near[n+val]]==False):
       
            giro[neartonear[n+val]]=near[n+val]
            if (checkCycle(giro,neartonear[val+n])==True):
               giro[neartonear[val+n]]=-1
               distances[neartonear[val+n],near[val+n]]=np.ma.masked
               neartonear[val+n]=np.ma.argmin(distances[:,near[val+n]])
               giro[neartonear[val+n]]=near[val+n]


            distances[neartonear[n+val]]=np.ma.masked
            distances[:,near[n+val]]=np.ma.masked
    
         elif (distances.mask[near[n+val],neartonear[n+val]]==False):
            giro[near[n+val]]=neartonear[n+val]
            if (checkCycle(giro,near[val+n])==True):
               giro[near[val+n]]=-1
               
               distances[near[val+n],neartonear[val+n]]=np.ma.masked
               neartonear[val+n]=np.ma.argmin(distances[near[val+n]])
               giro[near[val+n]]=neartonear[val+n]
            
          
            distances[near[n+val]]=np.ma.masked
            distances[:,neartonear[n+val]]=np.ma.masked


           
         
         distances[neartonear[n+val],near[n+val]]=np.ma.masked
         distances[near[n+val],neartonear[n+val]]=np.ma.masked

      if(onlybranch==0 and neartonear[1+n]==neartonear[n]):
         if not distances[near[1-val+n]].mask.all():
            neartonear[1-val+n]=np.ma.argmin(distances[near[1-val+n]])

      if(onlybranch!= 3 and onlybranch!=2):
         if (distances.mask[neartonear[1-val+n],near[1-val+n]]==False):

            giro[neartonear[1-val+n]]=near[1-val+n]

            if (checkCycle(giro,neartonear[1-val+n])==True):
               giro[neartonear[1-val+n]]=-1
               distances[neartonear[1-val+n],near[1-val+n]]=np.ma.masked
               neartonear[1-val+n]=np.ma.argmin(distances[:,near[1-val+n]])
               giro[neartonear[1-val+n]]=near[1-val+n]
               


            distances[neartonear[1-val+n]]=np.ma.masked
            distances[:,near[1-val+n]]=np.ma.masked
    
         elif (distances.mask[near[1-val+n],neartonear[1-val+n]]==False):

            giro[near[1-val+n]]=neartonear[1-val+n]
            if (checkCycle(giro,near[1-val+n])==True):
               giro[near[1-val+n]]=-1
               distances[near[1-val+n],neartonear[1-val+n]]=np.ma.masked
               neartonear[1-val+n]=np.ma.argmin(distances[near[1-val+n]])
               giro[near[1-val+n]]=neartonear[1-val+n]


            distances[near[1-val+n]]=np.ma.masked
            distances[:,neartonear[1-val+n]]=np.ma.masked

         distances[neartonear[1-val+n],near[1-val+n]]=np.ma.masked
         distances[near[1-val+n],neartonear[1-val+n]]=np.ma.masked
      

         

      n+=2
   
   return neartonear
   

   

def checkCycle(giro,start):  # check if there s a cycle , if there is a cycle and we haven't covered all vertex gives true otherwise gives false  
   togo=0
   step=giro[start]
   i=0
   while (step!=start and step>=0):
      i+=1
      step=giro[step]
      if step==-1:
         return False
   
   if(i<len(giro)-1):
      return True
   
   return False
   

   




def startgreedy2(start=-1):   # an other greedy approch , that down't work well ( but for small populations it's better then the other one )
   giro=[-1] * len(cities)

   distances = np.ma.masked_equal(DIST_MATRIX, np.inf)
   start_flat= np.argmax(distances)
   if( start==-1): #select as start and end the 2 furthest elements 
      start_2d = np.unravel_index(start_flat, distances.shape)
      start=start_2d[0]
      end=start_2d[1]
   else:   #select as  start the given one and as end the furthest from start 
      end=np.argmax(distances[start])


   #np.argmax(distances[start])

   startpoint=start #np.int64(np.random.randint(len(cities)))
   

   
   startvalue=-len(cities)
   near=[startvalue,startvalue,startvalue,startvalue]


   #end=np.ma.argmax(distances[startpoint])


   startexp3(startpoint,distances,False,near,giro)

   
   startexp3(end,distances,True,near,giro)

   count =giro.count(-1)
   while count >0:
      neartonear=findNearToNear(near,distances,giro)
      near=neartonear.copy()


      count =giro.count(-1)
 
   return giro



In [383]:
def devround(path,start):   #changes way in which the path is displayed , so it's the same as the one done using greedy1
    newpath=[]
    newpath.append(start)
    for e in range(len(path)):
        start=path[start]
        newpath.append(start)
   
   
    return newpath

Greedy part
Here i show the results of both greedy approches used, for more cities startgreedy would perform better then startgreedy2

In [384]:
totalcost=0

totalcostbasic=0

for i in range(1): #greedy approch starting from 1 
    
    totalcost+=tsp_cost(devround(startgreedy2(i),i))
    totalcostbasic+=tsp_cost(startgreedy(i))

ic(totalcost,totalcostbasic)





ic| totalcost: np.float64(1353.0818278087431)
    totalcostbasic: np.float64(1475.528091104531)


(np.float64(1353.0818278087431), np.float64(1475.528091104531))

Evolutionary part 

In [385]:
def tournament_sel(start,n=2): # tournament selection for deciding the nearest element    it's unused


    n = min(n, DIST_MATRIX.shape[1])

    # Choose n elements at random from the specified row
    random_elements_indexes = np.random.choice(DIST_MATRIX.shape[1], size=n, replace=True)
    random_elements_values= DIST_MATRIX[start][random_elements_indexes]



    bestIndex=np.argmin(random_elements_values)
    bestValue=random_elements_values[bestIndex]
    
    chosen=random_elements_indexes[bestIndex]

    return (start,chosen)


def tournament_sel_array(population,n=2): # tournament selection to decide which individual to crossover



    # Choose n elements at random from the specified row
    fitness_list=[]
    random_elements_indexes = np.random.choice(len(population), size=n, replace=True)
    for el in random_elements_indexes:
        fitness_list.append((population[el]).fitness)


    bestIndex=np.argmin(fitness_list)
    bestValue=fitness_list[bestIndex]
    
    chosen=population[random_elements_indexes[bestIndex]]

    return chosen





tournament_sel(0)

(0, np.int64(6))

In [386]:

def partially_mapped_crossover(parent1, parent2):
    size = len(parent1)
    # Step 1: Select crossover range at random
    start, end = sorted(random.sample(range(1, size - 2), 2))  # Avoid the first and last gene (the hive) (Last element of the list is (length - 1). Thus, it is (length - 2) to avoid the last gene)

    # Step 2: Create offspring by exchanging the selected range
    child1 = parent1[:start] + parent2[start:end] + parent1[end:]
    child2 = parent2[:start] + parent1[start:end] + parent2[end:]

    # Step 3: Determine the mapping relationship to legalize offspring
    mapping1 = {parent2[i]: parent1[i] for i in range(start, end)}
    mapping2 = {parent1[i]: parent2[i] for i in range(start, end)}

    # Step 4: Legalize children with the mapping relationship
    for i in list(range(start)) + list(range(end, size)):
        if child1[i] in mapping1:
            while child1[i] in mapping1:
                child1[i] = mapping1[child1[i]]
        if child2[i] in mapping2:
            while child2[i] in mapping2:
                child2[i] = mapping2[child2[i]]
    
    if random.random() < 0.5:
            
            return child1
    else:
            return child2





In [ ]:
@dataclass
class Individual:
    genome: np.ndarray
    fitness: float = None







def fitness(individual):  # lower fitness the better
    fit=0
    highvalue=300
    if(individual[0]!=individual[1]):
        fit+=len(individual)*highvalue
    
    if (set(individual) != set(range(len(cities)))):
        fit+=len(individual)*highvalue
    
    tot_cost = 0
    for c1, c2 in zip(individual, individual[1:]):
        fit += DIST_MATRIX[c1, c2]
   

    
    return fit

def popInizialize(): # inizialise the population using both tried greedy alghorithms 
    pop=[]

    
    dim=DIST_MATRIX.shape[1]
    nppl1=dim//3
    random_integers = np.random.choice(dim, size=nppl1, replace=False)
    for el in random_integers:
        pop.append(Individual(genome=startgreedy(el)))
    

    nppl2=dim//8
    random_integers = np.random.choice(dim, size=nppl2, replace=False)

    for el in random_integers:
        pop.append(Individual(genome=devround(startgreedy2(el),el)))

    
    for el in pop:
        el.fitness=fitness(el.genome)
    
    

    return pop

def inversion_mut(individual):   #Inversion Mutation implemntation 
    
    new_path = individual.genome.copy()

    # Randomly select two points
    idx1, idx2 = sorted(np.random.choice(len(new_path), 2, replace=False))
    # Invert the segment between idx1 and idx2
    new_path[idx1:idx2+1] = new_path[idx1:idx2+1][::-1]
    
   
    return Individual(genome=new_path,fitness=fitness(new_path))


    



history=[]

def EAlgoithm():
    
    mutrate=1
    pop=popInizialize()
    nstep=len(cities)*2
    lowest_fitness_individual = min(pop, key=lambda x: x.fitness)
    ic(lowest_fitness_individual.fitness)

    for el in tqdm(range(nstep)):
       
        if min(pop, key=lambda x: x.fitness).fitness==max(pop, key=lambda ind: ind.fitness).fitness:  #reached steady state
            break
        ris1=tournament_sel_array(pop,2)   # uses tournament selection to select 2 parent for the crossover
        ris2=tournament_sel_array(pop,2)
        
        child = partially_mapped_crossover(ris1.genome,ris2.genome)  # uses partially mapped crossover
        child=Individual(genome=child,fitness=fitness(child))
     
        if( np.random.rand()<mutrate):  # has a chance of mutating the child using inversion mutation
            child=inversion_mut(child)

        
        

        pop.append(child)  # add child to population
        
                

        lowest_fitness_individual = min(pop, key=lambda x: x.fitness)
        ic(lowest_fitness_individual.fitness)
        history.append(lowest_fitness_individual.fitness)
        
        maxfit_ind=(max(pop, key=lambda ind: ind.fitness))  
        ic(maxfit_ind.fitness)
        pop.remove(maxfit_ind) # delete from population the wrost individual
  

    return pop
    

pop=EAlgoithm()
minfit=min(pop, key=lambda x: x.fitness)
ic(minfit)


plt.figure(figsize=(14, 8))
plt.plot(
        range(len(history)),
        list(accumulate(history, min)),
        color="red",
    )
_ = plt.scatter(range(len(history)), history, marker=".")
    

    

ic| lowest_fitness_individual.fitness: np.float64(4053.0818278087427)
ic| lowest_fitness_individual.fitness: np.float64(4053.0818278087427)
ic| maxfit_ind.fitness: np.float64(4484.071940860687)
ic| lowest_fitness_individual.fitness: np.float64(4053.0818278087427)
ic| maxfit_ind.fitness: np.float64(4245.562474747089)
ic| lowest_fitness_individual.fitness: np.float64(4053.0818278087427)
ic| maxfit_ind.fitness: np.float64(4603.8945651377035)
ic| lowest_fitness_individual.fitness: np.float64(3991.730364572897)
ic| maxfit_ind.fitness: np.float64(4186.384417850251)
ic| lowest_fitness_individual.fitness: np.float64(3991.730364572897)
ic| maxfit_ind.fitness: np.float64(4424.114544898683)
ic| lowest_fitness_individual.fitness: np.float64(3991.730364572897)
ic| maxfit_ind.fitness: np.float64(4181.4725995315475)
ic| lowest_fitness_individual.fitness: np.float64(3991.730364572897)
ic| maxfit_ind.fitness: np.float64(4178.11502483949)
ic| lowest_fitness_individual.fitness: np.float64(3991.7303645728

KeyboardInterrupt: 